In [29]:
# Import the dependencies.
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests, time
from datetime import datetime
import sys
import gmaps
import gmaps.datasets
from config import g_api_key

In [6]:
# Import WeatherPY_challenge.csv in dataframe
file_to_import = os.path.join("data","WeatherPY_challenge.csv")

city_weather_df = pd.read_csv(file_to_import)
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall,Snowfall,Country,Date
0,Mataura,-46.19,168.86,55.60,66,48,6.38,scattered clouds,0.00,0.0,NZ,2020-05-06 03:18:42
1,Lichinga,-13.31,35.24,57.20,93,75,11.41,broken clouds,0.00,0.0,MZ,2020-05-06 03:18:42
2,Barrow,71.29,-156.79,17.60,78,1,11.41,clear sky,0.00,0.0,US,2020-05-06 03:18:43
3,Busselton,-33.65,115.33,55.00,94,67,26.75,light rain,0.14,0.0,AU,2020-05-06 03:18:43
4,Jamestown,42.10,-79.24,41.00,52,1,6.93,clear sky,0.00,0.0,US,2020-05-06 03:16:34
...,...,...,...,...,...,...,...,...,...,...,...,...
533,Novoilinsk,51.68,108.71,41.68,78,100,10.11,light rain,0.34,0.0,RU,2020-05-06 03:14:37
534,Auki,-8.77,160.70,85.59,64,73,5.06,broken clouds,0.00,0.0,SB,2020-05-06 03:14:37
535,Jaque,7.52,-78.17,81.05,85,100,2.10,light rain,0.28,0.0,PA,2020-05-06 03:14:37
536,Ketchikan,55.34,-131.65,53.60,58,1,9.17,clear sky,0.00,0.0,US,2020-05-06 03:14:38


In [47]:
## Ask the customer to add weather preferences: minimum and maximum temperature value, if raining or snowing.
min_temp = float(input("What is the minimum temperature you would like for your trip? (" + u"\N{DEGREE SIGN}" + "F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (" + u"\N{DEGREE SIGN}" + "F) "))

# Conditionals if it is a practical temperature for either snow or rain
if min_temp <=32 or max_temp <= 32:
    snowing_op = input("Do you want it to be snowing? (yes/no) ")
else:
    snowing_op = "no"
    
if min_temp >32 or max_temp >32:
    raining_op = input("Do you want it to be raining? (yes/no) ")
else:
    raining_op = "no"

What is the minimum temperature you would like for your trip? (°F) 65
What is the maximum temperature you would like for your trip? (°F) 85
Do you want it to be raining? (yes/no) yes


In [48]:
#setup criteria for the preferred city filters
if snowing_op == "yes":
    snow_criteria = city_weather_df["Snowfall"] > 0
else:
    snow_criteria = city_weather_df["Snowfall"] == 0

if raining_op == "yes":
    rain_criteria = city_weather_df["Rainfall"] > 0
else:
    rain_criteria = city_weather_df["Rainfall"] == 0
    
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) &
                                       (city_weather_df["Max Temp"] >= min_temp) & snow_criteria & rain_criteria].dropna()

print(len(preferred_cities_df))
preferred_cities_df.head(10)

25


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall,Snowfall,Country,Date
18,Atuona,-9.80,-139.03,81.07,76,13,17.11,light rain,0.18,0.0,PF,2020-05-06 03:18:47
61,Imbituba,-28.24,-48.67,66.20,93,98,2.24,light rain,0.48,0.0,BR,2020-05-06 03:11:58
66,Tura,25.52,90.22,81.90,67,57,3.96,light rain,0.81,0.0,IN,2020-05-06 03:12:03
88,Coruripe,-10.13,-36.18,77.94,86,67,7.87,light rain,0.12,0.0,BR,2020-05-06 03:08:42
199,Kavieng,-2.57,150.80,83.95,75,99,5.48,light rain,0.15,0.0,PG,2020-05-06 03:12:46
239,Lupiro,-8.38,36.67,70.93,94,97,2.28,light rain,0.72,0.0,TZ,2020-05-06 03:12:58
242,Airai,-8.93,125.41,73.76,80,31,3.74,light rain,0.35,0.0,TL,2020-05-06 03:12:59
244,Voh,-20.97,164.70,67.55,87,100,6.67,light rain,0.37,0.0,NC,2020-05-06 03:12:59
258,Sola,-13.88,167.55,82.13,72,94,7.78,light rain,0.78,0.0,VU,2020-05-06 03:13:04
270,Lorengau,-2.02,147.27,83.37,81,80,11.05,moderate rain,2.87,0.0,PG,2020-05-06 03:13:08


In [49]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_api_key}

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Weather Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

file_to_save = os.path.join("data","WeatherPy_vacation.csv")

pd.DataFrame(hotel_df).to_csv(file_to_save, index=False)


Hotel not found...skipping


In [58]:
# Setting up the info_box_template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_api_key)

# Add locations for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(10.0, 31.0), zoom_level=1.9)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))